In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from transformers import ViTModel
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from tqdm import tqdm
import time

# ==== MODEL ====

class CNN_ViT_FC(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN_ViT_FC, self).__init__()
        # EfficientNet CNN backbone
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn_features = nn.Sequential(*list(self.cnn.children())[:-1])  # remove classifier

        # ViT base
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224", add_pooling_layer=False)

        # Fully connected head
        self.fc = nn.Sequential(
            nn.Linear(1280 + 768, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        cnn_out = self.cnn_features(x)  # (B, 1280, 1, 1)
        cnn_out = cnn_out.flatten(1)    # (B, 1280)

        vit_out = self.vit(x).last_hidden_state[:, 0, :]  # CLS token: (B, 768)

        fused = torch.cat([cnn_out, vit_out], dim=1)  # (B, 2048)
        out = self.fc(fused)
        return out


# ==== DATA LOADER ====

def get_dataloaders(train_dir, test_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader


# ==== TRAINING LOOP ====

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4, save_path="cnn_vit_fc.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print(f"🔥 Best Val Accuracy: {best_acc:.4f}")


# ==== EVALUATION ====

def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total


# ==== MAIN RUN ====

if __name__ == "__main__":
    train_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Training"
    test_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Testing"
    batch_size = 8
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    model = CNN_ViT_FC(num_classes=2)
    train_loader, val_loader = get_dataloaders(train_dir, test_dir, batch_size)
    
    train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4, save_path="cnn_vit_fc.pth")


Epoch 1/50: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [01:16<00:00,  1.78it/s]


Epoch 1: Train Loss=60.9408, Train Acc=0.7980, Val Acc=0.7786
✅ Saved best model!


Epoch 2/50: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [01:14<00:00,  1.83it/s]


Epoch 2: Train Loss=48.3366, Train Acc=0.8478, Val Acc=0.7897
✅ Saved best model!


Epoch 3/50: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [01:14<00:00,  1.82it/s]


...

Epoch 48: Train Loss=4.8674, Train Acc=0.9889, Val Acc=0.8561


Epoch 49/50: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [01:14<00:00,  1.81it/s]


Epoch 49: Train Loss=3.4020, Train Acc=0.9935, Val Acc=0.8450


Epoch 50/50: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [01:14<00:00,  1.82it/s]


Epoch 50: Train Loss=7.1843, Train Acc=0.9834, Val Acc=0.8376
🔥 Best Val Accuracy: 0.8598


In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from transformers import ViTModel
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import os

# ==== MODEL ====

class OnlyViTClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(OnlyViTClassifier, self).__init__()
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224", add_pooling_layer=False)

        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        vit_out = self.vit(x).last_hidden_state[:, 0, :]  # CLS token
        return self.classifier(vit_out)


# ==== DATA LOADER ====

def get_dataloaders(train_dir, test_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader


# ==== TRAINING LOOP ====

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4, save_path="only_vit.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print(f"🔥 Best Val Accuracy: {best_acc:.4f}")


# ==== EVALUATION ====

def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total


# ==== MAIN RUN ====

if __name__ == "__main__":
    train_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Training"
    test_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Testing"
    batch_size = 16
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    model = OnlyViTClassifier(num_classes=2)
    train_loader, val_loader = get_dataloaders(train_dir, test_dir, batch_size)
    
    train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4, save_path="only_vit.pth")


/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch 1/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:00<00:00,  1.12it/s]


Epoch 1: Train Loss=33.0378, Train Acc=0.7768, Val Acc=0.7712
✅ Saved best model!


Epoch 2/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:00<00:00,  1.13it/s]


Epoch 2: Train Loss=22.6954, Train Acc=0.8589, Val Acc=0.6568


Epoch 3/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:00<00:00,  1.13it/s]


...

Epoch 48: Train Loss=0.0579, Train Acc=1.0000, Val Acc=0.8266


Epoch 49/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:01<00:00,  1.11it/s]


Epoch 49: Train Loss=0.0075, Train Acc=1.0000, Val Acc=0.8303


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:01<00:00,  1.11it/s]


Epoch 50: Train Loss=0.0424, Train Acc=1.0000, Val Acc=0.8303
🔥 Best Val Accuracy: 0.8487


In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# ==== MODEL ====

class OnlyEfficientNet(nn.Module):
    def __init__(self, num_classes=2):
        super(OnlyEfficientNet, self).__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.base_model.classifier[1].in_features, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

# ==== DATA LOADER ====

def get_dataloaders(train_dir, test_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader

# ==== TRAINING LOOP ====

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4, save_path="only_effnet.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print(f"🔥 Best Val Accuracy: {best_acc:.4f}")

# ==== EVALUATION ====

def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total

# ==== MAIN RUN ====

if __name__ == "__main__":
    train_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Training"
    test_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Testing"
    batch_size = 16
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    model = OnlyEfficientNet(num_classes=2)
    train_loader, val_loader = get_dataloaders(train_dir, test_dir, batch_size)
    
    train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4, save_path="only_effnet.pth")


/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:33<00:00,  2.05it/s]


Epoch 1: Train Loss=31.8937, Train Acc=0.7924, Val Acc=0.8413
✅ Saved best model!


Epoch 2/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:31<00:00,  2.16it/s]


Epoch 2: Train Loss=17.1897, Train Acc=0.9022, Val Acc=0.8635
✅ Saved best model!


Epoch 3/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:31<00:00,  2.17it/s]


...

Epoch 48: Train Loss=0.3449, Train Acc=0.9991, Val Acc=0.8561


Epoch 49/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:32<00:00,  2.12it/s]


Epoch 49: Train Loss=0.4248, Train Acc=0.9982, Val Acc=0.8561


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:32<00:00,  2.08it/s]


Epoch 50: Train Loss=1.2341, Train Acc=0.9945, Val Acc=0.8192
🔥 Best Val Accuracy: 0.8672


In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from transformers import ViTModel, ViTFeatureExtractor

# ==== MODEL ====

class CNNViTConcatNoAttention(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNViTConcatNoAttention, self).__init__()

        # CNN backbone (EfficientNet)
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()

        # ViT backbone (pretrained)
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

        # Feature dimensions
        cnn_out_dim = 1280
        vit_out_dim = 768

        # Final classifier
        self.fc = nn.Sequential(
            nn.Linear(cnn_out_dim + vit_out_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # CNN path
        cnn_features = self.cnn(x)  # (B, 1280)

        # ViT path
        vit_inputs = x.clone()
        vit_outputs = self.vit(pixel_values=vit_inputs, return_dict=True)
        vit_features = vit_outputs.pooler_output  # (B, 768)

        # Concatenate
        fused = torch.cat([cnn_features, vit_features], dim=1)  # (B, 2048)

        return self.fc(fused)

# ==== DATA LOADER ====

def get_dataloaders(train_dir, test_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    
    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader

# ==== TRAINING ====

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4, save_path="cnn_vit_concat_no_attn.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print(f"🔥 Best Val Accuracy: {best_acc:.4f}")

# ==== EVALUATION ====

def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total

# ==== MAIN ====

if __name__ == "__main__":
    train_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Training"
    test_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Testing"
    batch_size = 16
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    model = CNNViTConcatNoAttention(num_classes=2)
    train_loader, val_loader = get_dataloaders(train_dir, test_dir, batch_size)
    
    train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4, save_path="cnn_vit_concat_no_attn.pth")


/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
 

Epoch 1: Train Loss=27.9636, Train Acc=0.8054, Val Acc=0.8672
✅ Saved best model!


Epoch 2/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:14<00:00,  1.10s/it]


Epoch 2: Train Loss=17.4982, Train Acc=0.8801, Val Acc=0.8672


Epoch 3/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:15<00:00,  1.11s/it]


...

Epoch 48: Train Loss=0.5393, Train Acc=0.9982, Val Acc=0.8266


Epoch 49/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:11<00:00,  1.06s/it]


Epoch 49: Train Loss=0.1783, Train Acc=0.9991, Val Acc=0.7970


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:11<00:00,  1.06s/it]


Epoch 50: Train Loss=0.6550, Train Acc=0.9945, Val Acc=0.8303
🔥 Best Val Accuracy: 0.8672


In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTModel
from tqdm import tqdm

# ==== Self-Attention Block ====

class SelfAttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads=4):
        super(SelfAttentionBlock, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.ReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )

    def forward(self, x):
        attn_output, _ = self.attn(x, x, x)
        x = self.norm(x + attn_output)
        x = self.norm(x + self.ff(x))
        return x

# ==== Model ====

class CNNViTSelfAttention(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNViTSelfAttention, self).__init__()

        # CNN Backbone
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()
        self.cnn_pool = nn.AdaptiveAvgPool2d((1, 1))

        # ViT Backbone
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

        # Self-Attention on ViT output
        self.self_attn = SelfAttentionBlock(embed_dim=768, num_heads=4)

        # Final classifier
        self.classifier = nn.Sequential(
            nn.Linear(768 + 1280, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # CNN
        cnn_feat_map = self.cnn.features(x)                  # (B, 1280, H, W)
        cnn_pooled = self.cnn_pool(cnn_feat_map).squeeze(-1).squeeze(-1)  # (B, 1280)

        # ViT
        vit_out = self.vit(pixel_values=x, return_dict=True)
        vit_seq = vit_out.last_hidden_state                  # (B, N, 768)

        # Self-Attention over ViT sequence
        vit_self_attn = self.self_attn(vit_seq)              # (B, N, 768)
        vit_pooled = vit_self_attn[:, 0, :]                  # Take CLS token after self-attn

        # Concatenate
        fused = torch.cat([cnn_pooled, vit_pooled], dim=1)   # (B, 2048)
        return self.classifier(fused)

# ==== Data Loaders ====

def get_dataloaders(train_dir, test_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    
    train_dataset = ImageFolder(train_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader

# ==== Training ====

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4, save_path="cnn_vit_self_attn.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print(f"🔥 Best Val Accuracy: {best_acc:.4f}")

# ==== Evaluation ====

def evaluate_model(model, data_loader, device):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

# ==== Main ====

if __name__ == "__main__":
    train_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Training"
    test_dir = "/Users/ramanathanswaminathan/Downloads/glaucoma_exhaustive/acrima+origa/Testing"
    batch_size = 16
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

    model = CNNViTSelfAttention(num_classes=2)
    train_loader, val_loader = get_dataloaders(train_dir, test_dir, batch_size)

    train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4, save_path="cnn_vit_self_attn.pth")


/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramanathanswaminathan/miniconda3/envs/tf-metal-fix/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
 

Epoch 1: Train Loss=27.9916, Train Acc=0.7998, Val Acc=0.7970
✅ Saved best model!


Epoch 2/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:13<00:00,  1.08s/it]


Epoch 2: Train Loss=15.6495, Train Acc=0.8865, Val Acc=0.8192
✅ Saved best model!


Epoch 3/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:13<00:00,  1.08s/it]


...

Epoch 48: Train Loss=0.3046, Train Acc=0.9991, Val Acc=0.8487


Epoch 49/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:13<00:00,  1.08s/it]


Epoch 49: Train Loss=0.3071, Train Acc=0.9982, Val Acc=0.8229


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [01:12<00:00,  1.07s/it]


Epoch 50: Train Loss=0.3987, Train Acc=0.9982, Val Acc=0.8339
🔥 Best Val Accuracy: 0.8598
